In [ ]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

In [ ]:
using LinearAlgebra
using ForwardDiff

In [ ]:
function hat(v)
    return [0 -v[3] v[2];
            v[3] 0 -v[1];
            -v[2] v[1] 0]
end

In [ ]:
function L(q)
    s = q[1]
    v = q[2:4]
    L = [s    -v';
         v  s*I+hat(v)]
    return L
end

In [ ]:
function R(q)
    s = q[1]
    v = q[2:4]
    R = [s    -v';
         v  s*I-hat(v)]
    return R
end

In [ ]:
T = Diagonal([1; -ones(3)])
H = [zeros(1,3); I];

In [ ]:
function G(q)
    G = L(q)*H
end

function Q(q)
    return H'*(R(q)'*L(q))*H
end

In [ ]:
#Generate a random quaternion
qtrue = randn(4)
qtrue = qtrue/norm(qtrue)

Qtrue = Q(qtrue) #Generate equivalent rotation matrix

In [ ]:
#Generate data
vN = randn(3,10) #Generate some random world-frame vectors

#normalize
for k = 1:10
    vN[:,k] .= vN[:,k]./norm(vN[:,k])
end

vB = Qtrue'*vN #generate body-frame vectors

In [ ]:
function residual(q)
    r = vN - Q(q)*vB
    return r[:]
end

In [ ]:
#Random initial guess
q = randn(4)
q = q/norm(q)

In [ ]:
#Gauss-Newton Method
ϕ = ones(3)
iter = 0
while maximum(abs.(ϕ)) > 1e-8
    r = residual(q)
    dr = ForwardDiff.jacobian(residual, q)
    ∇r = dr*G(q)
    ϕ = -(∇r'*∇r)\(∇r'*r) #3-parameter update computed with gauss-newton
    q = L(q)*[sqrt(1-ϕ'*ϕ); ϕ] #multiplicative update applied to q
    iter += 1
end

In [ ]:
iter

In [ ]:
q-qtrue

In [ ]:
q+qtrue